## GA4 Reporting

This notebooks cover the creation of initial reporting tables for GA4 tracking data in this repository.  It also include the query used for incremental, daily, updates of the reporting table that take advantage of shared data from GA4 Exports and partitioningn of the reporting tables.